## current

In [4]:
# ============================================================
#  Step 0/8 : Colab Environment Setup
# ============================================================
# ⚠️  This block is intended for Google Colab development only.
#     When running as CLI (grader.py), this section will be skipped.
print("🔹 Step 0/8: Installing minimal dependencies and setting up environment...")

# ------------------------------------------------------------
# 0.1 Mount Google Drive
# ------------------------------------------------------------
from google.colab import drive

try:
    drive.mount("/content/drive")
    print("✅ Google Drive mounted successfully.")
except Exception as e:
    print(f"⚠️ Drive mount skipped or failed: {e}")

# ------------------------------------------------------------
# 0.2 Dependency check & install
# ------------------------------------------------------------
print("📦 Checking required libraries...")
%pip install -q nbformat==5.10.4 pandas==2.2.2

# ------------------------------------------------------------
# 0.3 Suppress warnings (Deprecation, Future)
# ------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
print("🔇 Deprecation and Future warnings are suppressed.")

# ------------------------------------------------------------
# 0.4 Set working directory
# ------------------------------------------------------------
import os, sys
PKG_PARENT = "/content/drive/MyDrive/Colab Notebooks/autograder/src"

try:
    os.chdir(PKG_PARENT)
    sys.path.append(PKG_PARENT)
    print(f"📁 Working directory set to: {PKG_PARENT}")
except Exception as e:
    print(f"⚠️ Directory change failed: {e}")


🔹 Step 0/8: Installing minimal dependencies and setting up environment...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive mounted successfully.
📦 Checking required libraries...
🔇 Deprecation and Future warnings are suppressed.
📁 Working directory set to: /content/drive/MyDrive/Colab Notebooks/autograder/src


In [ ]:
# ============================================================
#  Step 1/8 : Importing libraries & initializing
# ============================================================
print("🔹 Step 1/8: Importing libraries & initializing...")

# ------------------------------------------------------------
# 1.1. Third-party & stdlib
# ------------------------------------------------------------
from pathlib import Path
import pandas as pd

# ------------------------------------------------------------
# 1.2. AutoGrader internal modules
# ------------------------------------------------------------
from autograder.policy import (
    BASE_SCORE, PENALTY_REQUIRED_MISS, PENALTY_REQUIRED_MISMATCH, PENALTY_OPTIONAL_MISS,
    DEFAULT_TEMPLATE_SIM_THRESHOLD, DEFAULT_PAIR_SIM_THRESHOLD, score_rule_str
)
from autograder.io_utils import now_kst, load_config
from autograder.nb_utils import _sim, _label_key_robust
from autograder.paths import build_output_layout

from autograder.label_tagging import template_label_tagging, classify_labels
from autograder.grading import grade_submissions
from autograder.similarity import compute_similarity_pairs, build_similarity_df


from autograder.report import (
    load_prev_ids, compute_new_ids,
    RunConfigInput, compose_run_config,
    build_stats_block, render_run_summary, build_run_log_lines
)

# ------------------------------------------------------------
# 1.3. Load session config & define key paths
# ------------------------------------------------------------
# Select session & mode
session = load_config("autograder/configs/sessions.toml", 9, "DEV")
# session = load_config("autograder/configs/sessions.toml", 9, "PROD")

# Core paths
TEMPLATE_PATH = Path(session["template_path"])   # template notebook
ANSWER_PATH   = Path(session["answer_path"])     # answer notebook
SUBMIT_DIR    = Path(session["submit_dir"])      # submissions root
OUT_DIR       = Path(session["out_dir"])         # outputs root

# Tagged template & audit
TAGGED_TEMP_PATH = OUT_DIR / "tagged_template.ipynb"
TAG_AUDIT_PATH   = OUT_DIR / "tag_audit.csv"

# Execution output folder
RUN_TS   = now_kst().strftime("%Y%m%d_%H%M%S")
EXEC_DIR = OUT_DIR / "executed" / RUN_TS

# Output layout (all output paths centralized)
LAYOUT = build_output_layout(OUT_DIR, EXEC_DIR, RUN_TS)

# ============================================================
#  Step 2/8 : Tag template (required / optional_ex)
# ============================================================
print("🔹 Step 2/8: Tagging template (required / optional_ex) by labels...")

if not TAGGED_TEMP_PATH.exists():
    template_label_tagging(TEMPLATE_PATH, ANSWER_PATH, TAGGED_TEMP_PATH, TAG_AUDIT_PATH)
    print("✅ Tagged template created!")
else:
    print("ℹ️ Tagged template already exists.")

# ============================================================
#  Step 3/8 : Read tagged template & extract label info
# ============================================================
print("\n🔹 Step 3/8: Reading tagged template and answer (label-based)...")

(
    req_labels,
    opt_labels,
    req_idx,
    opt_idx,
    template_fp,
    required_cell_map,
    optional_cell_map,
) = classify_labels(TAGGED_TEMP_PATH)

# ============================================================
#  Step 4/8 : Thresholds & previous summary
# ============================================================
print("\n🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...")

TEMPLATE_SIM_THRESHOLD  = DEFAULT_TEMPLATE_SIM_THRESHOLD   # override as needed
PAIR_SIM_THRESHOLD      = DEFAULT_PAIR_SIM_THRESHOLD       # override as needed
ENABLE_SIMILARITY_CHECK = True

# Load student_id set from previous summary (see report.py)
prev_ids = load_prev_ids(OUT_DIR)

# ============================================================
#  Step 5/8 : Grade submissions
# ============================================================
print("\n🔹 Step 5/8: Collecting and grading submissions...")

submit_paths = sorted(Path(SUBMIT_DIR).rglob("*.ipynb"))

(
    summary_df,
    fps,
    sid2file,
    sid2name,
    sid2path,
    today_rows_tmp,
    EXCLUDED_REQ_ALL,
    EXCLUDED_OPT_ALL,
) = grade_submissions(
    submit_paths=submit_paths,
    template_path=TEMPLATE_PATH,
    answer_path=ANSWER_PATH,
    tagged_template_path=TAGGED_TEMP_PATH,
    req_labels=req_labels,
    opt_labels=opt_labels,
    template_fingerprint=template_fp,
    template_sim_threshold=TEMPLATE_SIM_THRESHOLD,
)

# ============================================================
#  Step 6/8 : Similarity check (optional)
# ============================================================
pairs, df_sim = [], pd.DataFrame()
if ENABLE_SIMILARITY_CHECK:
    print("\n🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...")
    pairs, df_sim = compute_similarity_pairs(
        fps=fps,
        sid2file=sid2file,
        sid2path=sid2path,
        sid2name=sid2name,
        sim_func=_sim,
        threshold=PAIR_SIM_THRESHOLD,
    )
else:
    print("\n⏩ Step 6/8: Similarity check skipped (ENABLE_SIMILARITY_CHECK=False).")
    # ensure df_sim exists with the right schema
    df_sim = build_similarity_df(df_sim.to_records(index=False) if not df_sim.empty else pairs)

# ============================================================
#  Step 7/8 : Save outputs
# ============================================================
print("\n🔹 Step 7/8: Saving outputs...")

# ------------------------------------------------------------
# 7.1. Summary CSVs
# ------------------------------------------------------------
summary_df.to_csv(LAYOUT.summary_ts, index=False, encoding="utf-8-sig")
summary_df.to_csv(LAYOUT.summary_latest, index=False, encoding="utf-8-sig")

# ------------------------------------------------------------
# 7.2. Similarity CSVs (optional)
# ------------------------------------------------------------

if not df_sim.empty:
    df_sim.to_csv(LAYOUT.similar_ts, index=False, encoding="utf-8-sig")
    df_sim.to_csv(LAYOUT.similar_latest, index=False, encoding="utf-8-sig")
# ------------------------------------------------------------
# 7.3. New-today CSVs
# ------------------------------------------------------------
df_today = pd.DataFrame(today_rows_tmp, columns=summary_df.columns)
df_today.to_csv(LAYOUT.newtoday_ts, index=False, encoding="utf-8-sig")
df_today.to_csv(LAYOUT.newtoday_latest, index=False, encoding="utf-8-sig")

# ============================================================
#  Step 8/8 : Log summary & print
# ============================================================
print("\n🔹 Step 8/8: Logging summary...")

_now = now_kst()
now_str = _now.strftime("%Y-%m-%d %H:%M:%S KST")
today_date = _now.date()

new_ids_sorted = compute_new_ids(summary_df, prev_ids)
STATS_BLOCK = build_stats_block(summary_df)

cfg_in = RunConfigInput(
    # time/run
    now_str=now_str, today_date=today_date, timezone="KST (UTC+9)", run_ts=RUN_TS,
    # paths
    out_dir=OUT_DIR, exec_dir=EXEC_DIR, submit_dir=SUBMIT_DIR,
    template_path=TEMPLATE_PATH, answer_path=ANSWER_PATH,
    tagged_temp_path=TAGGED_TEMP_PATH, tag_audit_path=TAG_AUDIT_PATH,
    summary_latest=LAYOUT.summary_latest,
    similar_latest=(LAYOUT.similar_latest if ENABLE_SIMILARITY_CHECK else None),
    newtoday_latest=LAYOUT.newtoday_latest,
    run_log_file=LAYOUT.run_log.name,  # CONFIG shows filename only

    # scoring / thresholds / options
    sim_threshold_template=TEMPLATE_SIM_THRESHOLD,
    sim_threshold_pair=PAIR_SIM_THRESHOLD,
    score_rule=score_rule_str(
        base=BASE_SCORE,
        req_miss=PENALTY_REQUIRED_MISS,
        req_mismatch=PENALTY_REQUIRED_MISMATCH,
        opt_miss=PENALTY_OPTIONAL_MISS,
    ),
    similarity_enabled=ENABLE_SIMILARITY_CHECK,

    # label info
    req_labels_count=len(req_labels), opt_labels_count=len(opt_labels),
    req_idx=req_idx, opt_idx=opt_idx,
    req_map=required_cell_map, opt_map=optional_cell_map,
    excluded_req_all=", ".join(sorted(EXCLUDED_REQ_ALL, key=_label_key_robust)) if EXCLUDED_REQ_ALL else "없음",
    excluded_opt_all=", ".join(sorted(EXCLUDED_OPT_ALL, key=_label_key_robust)) if EXCLUDED_OPT_ALL else "없음",

    # dataset stats
    total_cnt=len(summary_df), new_cnt=len(new_ids_sorted), today_cnt=len(df_today),
)

CONFIG = compose_run_config(cfg_in)

# Save run log
log_lines = build_run_log_lines(CONFIG, STATS_BLOCK, new_ids=new_ids_sorted)
with LAYOUT.run_log.open("a", encoding="utf-8") as f:
    f.write("\n".join(log_lines) + "\n")

print("🗒️ Log appended to:", CONFIG["RUN_LOG_FILE"])
print(render_run_summary(CONFIG, STATS_BLOCK))


🔹 Step 1/8: Importing libraries & initializing...
🔹 Step 2/8: Tagging template (required / optional_ex) by labels...
ℹ️ Tagged template already exists.

🔹 Step 3/8: Reading tagged template and answer (label-based)...

🔹 Step 4/8: Loading previous summary (if any) and setting thresholds...

🔹 Step 5/8: Collecting and grading submissions...

🔹 Step 6/8: Computing code similarity pairs (≥ 0.99)...
